В этом ноутбуке обогащаем исходный датасет данными из дополнительных источников

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 

from tqdm import tqdm

In [2]:
df = pd.read_csv('Data/reviews.csv')
df_mstore = pd.read_csv('Data/reviews_musicstore.csv')

In [3]:
df_mstore_filtered = df_mstore[df_mstore.rate!=5]
df_mstore_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 733 entries, 23 to 6024
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    733 non-null    int64 
 1   author        733 non-null    object
 2   date          733 non-null    object
 3   product_id    733 non-null    object
 4   product_name  733 non-null    object
 5   firm          733 non-null    object
 6   rate          733 non-null    int64 
 7   text          733 non-null    object
 8   topic         733 non-null    object
dtypes: int64(2), object(7)
memory usage: 57.3+ KB


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11214 entries, 0 to 11213
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    11214 non-null  int64 
 1   author        11209 non-null  object
 2   date          11214 non-null  object
 3   product_id    11214 non-null  int64 
 4   product_name  11214 non-null  object
 5   rate          11214 non-null  int64 
 6   text          11213 non-null  object
dtypes: int64(3), object(4)
memory usage: 613.4+ KB


In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df_mstore_filtered.drop(['Unnamed: 0', 'firm', 'topic'], axis=1, inplace=True)

C:\Users\Polina.DESKTOP-UGBOPLJ\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
df_full = pd.concat([df, df_mstore_filtered])

In [7]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11947 entries, 0 to 6024
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author        11942 non-null  object
 1   date          11947 non-null  object
 2   product_id    11947 non-null  object
 3   product_name  11947 non-null  object
 4   rate          11947 non-null  int64 
 5   text          11946 non-null  object
dtypes: int64(1), object(5)
memory usage: 653.4+ KB


In [8]:
df_full.isnull().sum()


author          5
date            0
product_id      0
product_name    0
rate            0
text            1
dtype: int64

In [9]:
df_full['author'] = df_full['author'].fillna('неизвестен') 

In [10]:
df_full = df_full.dropna()

In [11]:
df_full = df_full.drop_duplicates()

In [12]:
df_full.to_csv('Data/reviews_full_unprocessed.csv')

In [13]:
df_full.rate.value_counts()

5    7859
4    1381
3     435
0     186
2     165
1      87
Name: rate, dtype: int64

In [14]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10113 entries, 0 to 6024
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author        10113 non-null  object
 1   date          10113 non-null  object
 2   product_id    10113 non-null  object
 3   product_name  10113 non-null  object
 4   rate          10113 non-null  int64 
 5   text          10113 non-null  object
dtypes: int64(1), object(5)
memory usage: 553.1+ KB


In [15]:
df_reviews = df_full[['text', 'rate']]

In [61]:
!pip install mtranslate

  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3698 sha256=f715d5807ccc7576d575bf577de84cd7fedfd3c146fe96f2757e039d844c0452
  Stored in directory: c:\users\polina.desktop-ugboplj\appdata\local\pip\cache\wheels\99\49\c3\b1e6cb0d8b908238238a086ad2d0f30d4a7c692957e6cea74b
Successfully built mtranslate


In [62]:
from mtranslate import translate as translate_new

Идея в том, что после двойного перевода мы получим отзыв, несущий ту же смысловую нагрузку, но состоящий из других слов

In [ ]:
langs = [ 'es', 'pt', 'it', 'fr', 'ar', 'vi', 'ja', 'en']

In [68]:
def review_augmentator_new(phrase): # делает 2-ной перевод предложения и возвращает дубликат
    langs = ['de', 'es', 'pt', 'it', 'fr', 'ar', 'vi', 'ja', 'en']
    lang_sel = np.random.choice(langs)
    from_ru = translate_new(phrase,lang_sel,"auto")
    to_ru = translate_new(from_ru,'ru', 'auto')
    return {'text': to_ru, 'lang': lang_sel} 

In [ ]:
def review_augmentator(phrase): # делает 2-ной перевод предложения и возвращает дубликат
    langs = ['de', 'es', 'pt', 'it', 'fr', 'ar', 'vi', 'ja', 'en']
    lang_sel = np.random.choice(langs)
    from_ru = GT().translate(phrase, src='ru', dest=lang_sel).text
    to_ru = GT().translate(from_ru, src=lang_sel, dest='ru').text
    return {'text': to_ru, 'lang': lang_sel} 

In [74]:
test = df_full.text[10]
test

'Успел купить один такой микрофон до повышения цен за 3300. Качеством очень доволен! Записываю на него Hi-Hat и иногда использую для вокала (для живых выступлений). Верхушка чистая, звонкая, середина ровная. В общем, в этом микрофоне сочетаются все качества конденсаторного, но наряду\xa0\xa0этим - чувствительность как у динамического. Встроенный поп-фильтр весьма эффективный. За свои деньги микрофон огонь!'

In [75]:
review_augmentator_new(test)['text']

'Купил такой микрофон до повышения цены до 3300. Качеством очень доволен! Я записываю на него хай-хэт и иногда использую его для вокала (для живых выступлений). Верх чистый, блестящий, середина ровная. В целом этот микрофон сочетает в себе все качества конденсаторного, но вместе с этим - чувствительность электродинамического устройства. Встроенный поп-фильтр очень эффективен. За свои деньги огонь микро!'

In [71]:
def df_foreign_dublicator(df, y, types=None):
    df = df.reset_index(drop=True)
    y = y.reset_index(drop=True)
    if not types:
        types = y.unique()
    for rtype in types:
        rtype_indx_list = []
        for i,el in enumerate(y):
            if el == rtype:
                rtype_indx_list.append(i)
        iter_indx = tqdm(rtype_indx_list)
        for i in iter_indx:
            augmentator = review_augmentator_new(df[i])
            iter_indx.set_description('type: "{}"; lang: "ru-{}-ru" '.format(rtype, augmentator['lang']), refresh=True)
            df = df.append(pd.Series(augmentator['text']), ignore_index=True)
            y = y.append(pd.Series(rtype), ignore_index=True)
            
    return (df, y)

In [ ]:
# X, y = df_foreign_dublicator(X, y, types=['bad', 'neutral'])
# pd.concat([X, y], axis=1, ignore_index=True).to_csv('Data/after_augmentation.csv', index=False, header=['review_text', 'review_type'])

In [32]:
for rtype in types:
    rtype_indx_list = []
    for i,el in enumerate(y):
        if el == rtype:
            rtype_indx_list.append(i)

In [76]:
X, y = df_foreign_dublicator(df_reviews.text, df_reviews.rate, types=[0,1,2,3,4])

type: "4"; lang: "ru-vi-ru" : 100%|████████████████████████████████████████████████| 1381/1381 [20:33<00:00,  1.12it/s]


In [77]:
df_augmented = pd.concat([X, y], axis=1, ignore_index=True)
df_augmented.columns = ['text', 'rate']

In [79]:
df_augmented.tail(10)

,text,rate
12357,Купил в качестве запаски - цена была завышена....,4
12358,Хорошее соотношение цены и качества.,4
12359,"Думаю, это был спонтанный ход, и мне этот моду...",4
12360,"Большой красивый матовый рулон. Одна проблема,...",4
12361,Функция петли в сочетании с длинным затуханием...,4
12362,"Заказал по ошибке, хотел подстаканник для микр...",4
12363,"Я живу в России, эти частоты работают хорошо, ...",4
12364,Ты можешь взять это!,4
12365,"Погнут кабель, но работает нормально",4
12366,За свою цену эти мониторы - небольшой подарок....,4


In [80]:
df_augmented.rate.value_counts()

5    7859
4    2762
3     870
0     372
2     330
1     174
Name: rate, dtype: int64

In [82]:
df_augmented = df_augmented.drop_duplicates()

In [83]:
df_augmented.rate.value_counts()

5    7791
4    2696
3     860
0     366
2     328
1     174
Name: rate, dtype: int64

In [84]:
df_augmented.to_csv('Data/after_augmentation.csv', index=False)

In [85]:
df_eng = pd.read_csv('Data/Musical_instruments_reviews_Amazon.csv')

In [86]:
df_eng.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [87]:
df_eng = df_eng[['reviewText', 'overall']]

In [88]:
df_eng = df_eng[df_eng.overall<5]

In [89]:
df_eng = df_eng.dropna()

In [90]:
df_eng.overall.value_counts()

4.0    2083
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

In [91]:
def translate_to_ru(phrase):
    return translate_new(phrase, 'ru', 'auto')

In [94]:
translate_to_ru('I now use this cable to run from the output')

'Теперь я использую этот кабель для запуска от выхода'

In [95]:
df_eng['text'] = df_eng.reviewText.apply(translate_to_ru)

In [96]:
df_eng.head()

,reviewText,overall,text
7,I now use this cable to run from the output of...,3.0,Теперь я использую этот кабель для перехода от...
11,I got it to have it if I needed it. I have fou...,4.0,"Я получил его, если мне это нужно. Я обнаружил..."
12,If you are not use to using a large sustaining...,3.0,Если вы не привыкли использовать большую педал...
15,I bought this to use with my keyboard. I wasn'...,2.0,"Я купил это, чтобы использовать со своей клави..."
16,This Fender cable is the perfect length for me...,4.0,Этот кабель Fender - идеальная длина для меня!...


In [97]:
df_eng = df_eng[['text', 'overall']]
df_eng.columns = ['text', 'rate']
df_eng.head()

,text,rate
7,Теперь я использую этот кабель для перехода от...,3.0
11,"Я получил его, если мне это нужно. Я обнаружил...",4.0
12,Если вы не привыкли использовать большую педал...,3.0
15,"Я купил это, чтобы использовать со своей клави...",2.0
16,Этот кабель Fender - идеальная длина для меня!...,4.0


In [98]:
df_eng['rate'] = df_eng['rate'].astype(int)
df_eng.head()

,text,rate
7,Теперь я использую этот кабель для перехода от...,3
11,"Я получил его, если мне это нужно. Я обнаружил...",4
12,Если вы не привыкли использовать большую педал...,3
15,"Я купил это, чтобы использовать со своей клави...",2
16,Этот кабель Fender - идеальная длина для меня!...,4


In [99]:
df_new = pd.concat([df_augmented, df_eng])

In [100]:
df_new.rate.value_counts()

5    7791
4    4779
3    1632
2     578
1     391
0     366
Name: rate, dtype: int64

In [102]:
df_new = df_new.drop_duplicates()

In [103]:
df_new.rate.value_counts()

5    7791
4    4778
3    1630
2     577
1     391
0     366
Name: rate, dtype: int64

In [104]:
df_new.to_csv('Data/after_aug_full.csv', index=False)